In [ ]:
# @title Environment Setup
import os
import sys
if 'MPLBACKEND' in os.environ:
    del os.environ['MPLBACKEND']
    print("MPLBACKEND environment variable cleared.")

# 2. Clone the repository
!rm -rf FontDiffusion
!git clone https://github.com/dzungphieuluuky/FontDiffusion.git

In [ ]:
import os
import sys
from IPython import get_ipython
from typing import Optional

def configure_environment_paths():
    try:
        if "google.colab" in str(get_ipython()):
            print("✅ Environment: Google Colab")
            base_data_path = "/content/"
            base_output_path = "/content/"
            environment_name = "colab"
        elif os.environ.get("KAGGLE_KERNEL_RUN_TYPE"):
            print("✅ Environment: Kaggle")
            base_data_path = "/kaggle/input/"
            base_output_path = "/kaggle/working/"
            environment_name = "kaggle"
        else:
            print("⚠️ Environment: Local/Unknown")
            base_data_path = "./data/"
            base_output_path = "./output/"
            environment_name = "local"
    except NameError:
        print("⚠️ Non-interactive session. Using local paths.")
        base_data_path = "./data/"
        base_output_path = "./output/"
        environment_name = "local"
    os.makedirs(base_output_path, exist_ok=True)
    print(f"📂 Data Path: {base_data_path}")
    print(f"📦 Output Path: {base_output_path}")
    return base_data_path, base_output_path, environment_name

def load_secret(key_name: str) -> Optional[str]:
    env = ENV_NAME
    secret_value = None
    print(f"Attempting to load secret '{key_name}' from '{env}' environment...")
    try:
        if env == "colab":
            from google.colab import userdata
            secret_value = userdata.get(key_name)
        elif env == "kaggle":
            from kaggle_secrets import UserSecretsClient
            user_secrets = UserSecretsClient()
            secret_value = user_secrets.get_secret(key_name)
        else:
            secret_value = os.getenv(key_name)
        if not secret_value:
            print(f"⚠️ Secret '{key_name}' not found in the {env} environment.")
            return None
        print(f"✅ Successfully loaded secret '{key_name}'.")
        return secret_value
    except Exception as e:
        print(f"❌ An error occurred while loading secret '{key_name}': {e}")
        return None

def print_system_info():
    print("\n🔧 System Information")
    print(f"Python version: {sys.version.split()[0]}")
    try:
        import torch
        print(f"PyTorch version: {torch.__version__}")
        if torch.cuda.is_available():
            print(f"CUDA version: {torch.version.cuda}")
            print(f"GPU count: {torch.cuda.device_count()}")
            for i in range(torch.cuda.device_count()):
                print(f"  GPU {i}: {torch.cuda.get_device_name(i)}")
        else:
            print("CUDA not available")
    except ImportError:
        print("PyTorch not installed")
    finally:
      !nvidia-smi

INPUT_PATH, OUTPUT_PATH, ENV_NAME = configure_environment_paths()
is_kaggle = ("kaggle" in ENV_NAME)
is_colab = not is_kaggle
print_system_info()

os.environ["WANDB_API_KEY"] = wandb_key = load_secret("WANDB_API_KEY")
os.environ["HF_TOKEN"] = HF_TOKEN = load_secret('HF_TOKEN')

# Now, these libraries will log in automatically
import wandb
import huggingface_hub

wandb.login() 
huggingface_hub.login(token=os.environ["HF_TOKEN"]) 

In [ ]:
!uv pip install --upgrade pip
# 3. Install PyTorch 1.13
%cd {OUTPUT_PATH}
# Force reinstall torch 1.13 to match the model's training environment
# !uv pip uninstall torch torchvision
# !uv pip install torch==1.13.1+cu117 torchvision==0.14.1+cu117 --extra-index-url https://download.pytorch.org/whl/cu117
!uv pip install torch==2.9 torchvision
# 4. Install other dependencies

print("\n⬇️ Installing Dependencies (Manually fixed)...")
# Install xformers compatible with Torch 1.13
!uv pip install xformers==0.0.16 -q

# Install original dependencies
!uv pip install transformers==4.33.1 accelerate==0.23.0 diffusers==0.22.0
!uv pip install gradio==4.8.0 pyyaml pygame opencv-python info-nce-pytorch kornia
# -----------------------------------------------------------------
!uv pip install lpips scikit-image pytorch-fid
!sudo apt-get update && sudo apt-get install dos2unix
!uv pip install gdown tqdm
!uv pip install wandb
!uv pip install --upgrade pyarrow datasets
print("\n✅ Environment setup complete. You can now proceed to Block 2 (Inference).")

In [ ]:
# KAGGLE CELL #1: Download checkpoint
if is_colab:
  !uv pip install --upgrade "huggingface-hub>=0.34.0,<1.0"
else:
  !uv pip install --upgrade "huggingface-hub==0.25.2" "protobuf<5.0.0" "numpy<2.0.0"
import os
import sys
from tqdm.auto import tqdm
from pathlib import Path
os.chdir(OUTPUT_PATH)
# Download from Hub
if not os.path.exists("ckpt") or not list(Path("ckpt").glob("*.safetensors")):
    print("📥 Downloading checkpoint from Hugging Face Hub...\n")
    from huggingface_hub import snapshot_download
    snapshot_download(
        repo_id="dzungpham/font-diffusion-weights",
        local_dir="ckpt",
        allow_patterns="*.safetensors",
        force_download=False
    )
    print("\n✅ Download complete!")
else:
    print("✅ Checkpoint already downloaded")
# Verify
print("\n📂 Files in ckpt/:")
for file in os.listdir("ckpt"):
    if file.endswith(".safetensors"):
        size = os.path.getsize(f"ckpt/{file}") / (1024**2)
        print(f"  ✓ {file} ({size:.2f} MB)")

In [ ]:
# @title Unzipping all archived files
import os
import glob
from zipfile import ZipFile

zip_file_paths = glob.glob(os.path.join(INPUT_PATH, '*.zip'))

if not zip_file_paths:
    print(f'No .zip files found in {INPUT_PATH}.')
else:
    for zip_file_path in zip_file_paths:
        if os.path.exists(zip_file_path):
            print(f'Unzipping {zip_file_path}...')
            !unzip -o {zip_file_path} -d ./
            print(f'Unzipping of {zip_file_path} complete.')
        else:
            print(f'Error: The file {zip_file_path} was not found (post-glob check).')

In [ ]:
import pandas as pd
import os
def convert_csv_to_chars_txt(input_csv_path: str, output_txt_path: str, column_name: str = 'word'):
    """
    Reads a CSV file, extracts text from a specified column, and writes each character
    to a new line in a plain text file.
    Args:
        input_csv_path (str): The full path to the input CSV file.
        output_txt_path (str): The full path for the output text file.
        column_name (str): The name of the column in the CSV file containing the text.
    """
    if not os.path.exists(input_csv_path):
        print(f"Error: Input CSV file not found at '{input_csv_path}'. Please ensure the file is uploaded.")
        return
    try:
        df = pd.read_csv(input_csv_path)
    except Exception as e:
        print(f"Error reading CSV file '{input_csv_path}': {e}")
        return
    if column_name not in df.columns:
        print(f"Error: Column '{column_name}' not found in the CSV file '{input_csv_path}'.")
        return
    all_characters = []
    for item in df[column_name].astype(str).dropna().tolist():
        for char in item:
            all_characters.append(char)
    os.makedirs(os.path.dirname(output_txt_path), exist_ok=True)
    with open(output_txt_path, "w", encoding="utf-8") as f:
        f.write("\n".join(all_characters))
    print(f"Successfully converted '{input_csv_path}' to '{output_txt_path}', with one character per line.")
print("\n--- Demonstrating function with a dummy CSV file ---")
dummy_csv_path = os.path.join(OUTPUT_PATH, "dummy_data.csv")
dummy_output_txt_path = os.path.join(OUTPUT_PATH, "dummy_chars.txt")
dummy_data = {'word': ['hello', 'world', 'python']}
pd.DataFrame(dummy_data).to_csv(dummy_csv_path, index=False)
print(f"Created a dummy CSV file at: {dummy_csv_path}")
convert_csv_to_chars_txt(dummy_csv_path, dummy_output_txt_path)

In [ ]:
print("Model files:")
!ls -larth {OUTPUT_PATH}/ckpt

In [ ]:
%cd {OUTPUT_PATH}
# ==========================================
# EXPORT / DOWNLOAD DATASET COMMANDS
# ==========================================
HF_USERNAME = "dzungpham"
# Train Split
!python FontDiffusion/export_hf_dataset_to_disk.py \
  --output_dir "my_dataset/train_original" \
  --repo_id {HF_USERNAME}/font-diffusion-generated-data \
  --split "train_original" \
  --token HF_TOKEN

!python FontDiffusion/export_hf_dataset_to_disk.py \
  --output_dir "my_dataset/train" \
  --repo_id {HF_USERNAME}/font-diffusion-generated-data \
  --split "train" \
  --token HF_TOKEN
# Validation: Unseen Both
!python FontDiffusion/export_hf_dataset_to_disk.py \
  --output_dir "my_dataset/val" \
  --repo_id {HF_USERNAME}/font-diffusion-generated-data \
  --split "val" \
  --token HF_TOKEN

In [ ]:
print("Fonts currently in fonts/ folder")
!ls -lt FontDiffusion/fonts
print("Styles in style_images/ folder")
!ls -l FontDiffusion/styles_images

In [ ]:
import json
from pathlib import Path
from typing import Set

def remove_unparseable_from_checkpoint(checkpoint_path: str, unparseable_txt_path: str) -> None:
    """
    Removes generations referencing unparseable files from a results_checkpoint.json file.

    Args:
        checkpoint_path (str): Path to results_checkpoint.json.
        unparseable_txt_path (str): Path to unparseable_files.txt (absolute paths, one per line).
    """
    # Load unparseable file names into a set
    with open(unparseable_txt_path, "r", encoding="utf-8") as f:
        unparseable_files: Set[str] = {Path(line.strip()).name for line in f if line.strip()}

    # Load checkpoint JSON
    with open(checkpoint_path, "r", encoding="utf-8") as f:
        results = json.load(f)

    generations = results.get("generations", [])
    original_count = len(generations)

    # Filter out generations whose target_image_path's filename is in unparseable_files
    filtered_generations = [
        gen for gen in generations
        if os.path.join("kaggle/working/my_dataset/train_original", gen.get("target_image_path", "")) not in unparseable_files
    ]

    removed_count = original_count - len(filtered_generations)
    results["generations"] = filtered_generations

    # Save updated checkpoint
    with open(checkpoint_path, "w", encoding="utf-8") as f:
        json.dump(results, f, indent=2, ensure_ascii=False)

    print(f"Removed {removed_count} generations from {checkpoint_path}.")

# Example usage:
remove_unparseable_from_checkpoint(
    "my_dataset/train_original/results_checkpoint.json",
    "my_dataset/unparseable_files.txt"
)

In [ ]:
!cat my_dataset/unparseable_files.txt

In [ ]:
if is_colab:
  !uv pip install --upgrade "huggingface-hub>=0.34.0,<1.0"
else:
  !uv pip install --upgrade "huggingface-hub==0.25.2" "protobuf<5.0.0" "numpy<2.0.0"
%cd {OUTPUT_PATH}
!accelerate launch --num_processes 1 \
    FontDiffusion/sample_batch.py \
    --characters "FontDiffusion/NomTuTao/Ds_10k_ChuNom_TuTao.txt" \
    --style_images "FontDiffusion/styles_images" \
    --ckpt_dir "ckpt/" \
    --ttf_path "FontDiffusion/fonts/NomNaTong-Regular.otf" \
    --output_dir "my_dataset/train_original" \
    --num_inference_steps 20 \
    --guidance_scale 7.5 \
    --start_line 301 \
    --end_line 600 \
    --batch_size 35 \
    --save_interval 1 \
    --channels_last \
    --seed 42 \
    --compile \
    --enable_xformers

In [ ]:
!find my_dataset/train_original/ContentImage -type f | wc -l
!find my_dataset/train_original/TargetImage -type f | wc -l

In [ ]:
# !ls -lt my_dataset/train_original/ContentImage/*
# !ls -l my_dataset/train_original/TargetImage/*hanhthu*
# !ls -lt my_dataset/train_original/TargetImage/*

In [ ]:
import re
from pathlib import Path

# Your valid pattern
expected_pattern = r"U\+[0-9A-F]{4,5}.*_[0-9a-f]{8}\.png"

# Define the root directory ('.' for current directory)
root_dir = Path('./my_dataset/train_original/TargetImage')

# .rglob('*') finds every file recursively
for path in root_dir.rglob('*'):
    # Process only files (ignore directories)
    if path.is_file():
        # Check if the FILENAME (path.name) matches the regex
        if not re.match(expected_pattern, path.name):
            try:
                print(f"Deleting invalid file: {path}")
                # path.unlink() # This deletes the file
            except Exception as e:
                print(f"Error deleting {path}: {e}")

In [17]:
!python FontDiffusion/create_validation_split.py \
  --data_root my_dataset \
  --val_ratio 0.2 \
  --seed 42

2026-01-02 04:39:25,499 | INFO | ✓ Using source directory: my_dataset/train_original
2026-01-02 04:39:25,504 | INFO | 
2026-01-02 04:39:25,504 | INFO | FONTDIFFUSION VALIDATION SPLIT CREATOR
2026-01-02 04:39:25,504 | INFO | ============================================================
2026-01-02 04:39:25,504 | INFO | 
2026-01-02 04:39:25,504 | INFO | ANALYZING TRAINING DATA
2026-01-02 04:39:25,504 | INFO | ============================================================
2026-01-02 04:39:25,504 | INFO | 
🔍 Scanning content images...
Content images: 100%|████████████████████| 599/599 [00:00<00:00, 482687.43img/s]
2026-01-02 04:39:25,509 | INFO |   ✓ Found 599 content images
2026-01-02 04:39:25,510 | INFO | 
🔍 Scanning target images...
Styles: 100%|███████████████████████████████| 15/15 [00:00<00:00, 343.89style/s]
2026-01-02 04:39:25,554 | INFO |   ✓ Found 8985 valid target images
2026-01-02 04:39:25,554 | INFO | 
🔍 Validating content ↔ target pairs...
Validating pairs: 100%|█████████████████

In [ ]:
!uv pip install --upgrade pyarrow datasets

In [ ]:
# remove_unparseable_files.py

with open("my_dataset/unparseable_files.txt", "r", encoding="utf-8") as f:
    paths = [line.strip() for line in f if line.strip()]

import os

for path in paths:
    try:
        if os.path.exists(path):
            os.remove(path)
            print(f"Deleted: {path}")
        else:
            print(f"Not found: {path}")
    except Exception as e:
        print(f"Error deleting {path}: {e}")

In [18]:
HF_USERNAME = "dzungpham"
!python FontDiffusion/create_hf_dataset.py \
  --data_dir "my_dataset/train_original" \
  --repo_id dzungpham/font-diffusion-generated-data \
  --split "train_original" \
  --token {HF_TOKEN}

# Train Split
!python FontDiffusion/create_hf_dataset.py \
  --data_dir "my_dataset/train" \
  --repo_id dzungpham/font-diffusion-generated-data \
  --split "train" \
  --token {HF_TOKEN}

# Train Split
!python FontDiffusion/create_hf_dataset.py \
  --data_dir "my_dataset/val" \
  --repo_id dzungpham/font-diffusion-generated-data \
  --split "val" \
  --token {HF_TOKEN}



FONTDIFFUSION DATASET CREATOR

Data dir: my_dataset/train_original
Repo: dzungpham/font-diffusion-generated-data
Push to Hub: True
✓ Validated directory structure
  Content images: my_dataset/train_original/ContentImage
  Target images: my_dataset/train_original/TargetImage
  Results checkpoint: my_dataset/train_original/results_checkpoint.json

BUILDING DATASET

✓ Loaded results_checkpoint.json
  Generations: 8985
  Characters: 599
  Styles: 15

🖼️  Loading 8985 image pairs...
Loading image pairs: 100%|██████████████| 8.98k/8.98k [00:07<00:00, 1.18kpair/s]
✓ Loaded 8985 samples

PUSHING TO HUB
Repository: dzungpham/font-diffusion-generated-data
Split: train_original
Map: 100%|████████████████████████| 8985/8985 [00:00<00:00, 15307.79 examples/s]

Uploading the dataset shards: 100%|██████████| 1/1 [00:01<00:00,  1.25s/ shards]

✓ Successfully pushed to Hub!
  URL: https://huggingface.co/datasets/dzungpham/font-diffusion-generated-data

✅ COMPLETE!

FONTDIFFUSION DATASET CREATOR

Data 

In [ ]:
import torch, gc
torch.cuda.empty_cache()
gc.collect()

In [21]:
# TRAINING PHASE 1
if is_colab:
  !uv pip install --upgrade "huggingface-hub>=0.34.0,<1.0"
else:
  !uv pip install --upgrade "huggingface-hub==0.25.2"
import wandb

!accelerate launch FontDiffusion/train.py \
    --seed=123 \
    --experience_name="FontDiffuser_training_phase_1" \
    --data_root="my_dataset" \
    --output_dir="outputs/FontDiffuser" \
    --report_to="wandb" \
    --resolution=96 \
    --style_image_size=96 \
    --content_image_size=96 \
    --content_encoder_downsample_size=3 \
    --channel_attn=True \
    --content_start_channel=64 \
    --style_start_channel=64 \
    --train_batch_size=8 \
    --gradient_accumulation_steps=2 \
    --perceptual_coefficient=0.05 \
    --offset_coefficient=0.7 \
    --max_train_steps=200 \
    --ckpt_interval=100 \
    --log_interval=50 \
    --learning_rate=1e-4 \
    --lr_scheduler="linear" \
    --lr_warmup_steps=10000 \
    --drop_prob=0.1 \
    --mixed_precision="no"

Using Python 3.11.13 environment at: /usr
Resolved 12 packages in 78ms                                         
Audited 12 packages in 0.12ms
/usr/local/lib/python3.11/dist-packages/transformers/utils/generic.py:311: FutureWarning: `torch.utils._pytree._register_pytree_node` is deprecated. Please use `torch.utils._pytree.register_pytree_node` instead.
  torch.utils._pytree._register_pytree_node(
/usr/local/lib/python3.11/dist-packages/pydantic/_internal/_generate_schema.py:2249: UnsupportedFieldAttributeWarning: The 'repr' attribute with value False was provided to the `Field()` function, which has no effect in the context it was used. 'repr' is field-specific metadata, and can only be attached to a model field using `Annotated` metadata or by assignment. This may have happened because an `Annotated` type alias using the `type` statement was used, or if the `Field()` function was attached to a single member of a union type.
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/pydan

In [ ]:
!ls -lr outputs/FontDiffuser

In [ ]:
# TRAINING PHASE 2
!wandb login
!python FontDiffusion/my_train.py \
    --seed=123 \
    --experience_name="FontDiffuser_training_phase_2" \
    --data_root="my_dataset" \
    --output_dir="outputs/FontDiffuser" \
    --report_to="wandb" \
    --phase_2 \
    --phase_1_ckpt_dir="outputs/FontDiffuser/global_step_2000" \
    --scr_ckpt_path="ckpt/scr_210000.pth" \
    --sc_coefficient=0.05 \
    --num_neg=13 \
    --resolution=96 \
    --style_image_size=96 \
    --content_image_size=96 \
    --content_encoder_downsample_size=3 \
    --channel_attn=True \
    --content_start_channel=64 \
    --style_start_channel=64 \
    --train_batch_size=8 \
    --perceptual_coefficient=0.03 \
    --offset_coefficient=0.4 \
    --max_train_steps=100 \
    --ckpt_interval=50 \
    --gradient_accumulation_steps=2 \
    --log_interval=50 \
    --learning_rate=1e-5 \
    --lr_scheduler="constant" \
    --lr_warmup_steps=1000 \
    --drop_prob=0.1 \
    --mixed_precision="no"


In [ ]:
!python FontDiffusion/pth2safetensors.py \
    --weights_dir "ckpt" \
    --repo_id "dzungpham/font-diffusion-weights" \
    --token "{HF_TOKEN}"

In [ ]:
import os
import zipfile
from pathlib import Path
from typing import List
def find_result_folders(base_path: Path, pattern_name: str) -> List[Path]:
    return [p for p in base_path.glob(pattern_name) if p.is_dir()]

def zip_folder(folder_path: Path, output_base_path: Path) -> bool:
    folder_name = folder_path.name
    zip_path = output_base_path / f"{folder_name}.zip"
    try:
        print(f"   -> Zipping folder: {folder_name}...")
        with zipfile.ZipFile(zip_path, mode="w", compression=zipfile.ZIP_DEFLATED) as zipf:
            for file_path in folder_path.rglob("*"):
                if file_path.is_file():
                    arcname = file_path.relative_to(folder_path.parent)
                    zipf.write(file_path, arcname)
        print(f"   ✅ Created ZIP: {zip_path.name}")
        return True
    except Exception as exc:
        print(f"   ❌ Failed to zip {folder_name}: {exc}")
        return False

def zip_stats_results_folders(output_base_path: str, pattern_name: str) -> None:
    base = Path(output_base_path)
    base.mkdir(parents=True, exist_ok=True)
    result_folders = find_result_folders(base, pattern_name)
    if not result_folders:
        print(f"⚠️ No folders matching '*dataset' found in '{output_base_path}'.")
        return
    print(f"🔍 Found {len(result_folders)} result folder(s) to zip.")
    successful = sum(1 for folder in result_folders if zip_folder(folder, base))
    print(f"\n✅ DONE! Successfully zipped {successful} out of {len(result_folders)} folder(s).")

if __name__ == "__main__":
    try:
        output_root = os.getenv("OUTPUT_PATH") or globals().get("OUTPUT_PATH")
        if not output_root:
            raise ValueError("OUTPUT_PATH not defined")
        zip_stats_results_folders(
            output_base_path=OUTPUT_PATH,
            pattern_name="my_dataset")
    except Exception as e:
        print(f"❌ An error occurred: {e}")